Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import seaborn as sns
import re
%matplotlib inline
pd.options.display.max_colwidth = 750

In [2]:
# You may need to change the path
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133",1
1,3861,"\nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)",0
2,655,"\nThis release is a port version of Amrut’s Intermediate Sherry — a sort of port pipe sandwich. The spirit is matured in both unused casks and bourbon casks, then spends a few months in port pipes, and then returns to bourbon casks. The result is a Pink Floyd show of a whisky: vibrant, colorful, complex, and nearly too much. A blackcurrant and wispy, smoky nose gives way to an intense and bittersweet mix of chili, blackcurrant, oak, damson, dark chocolate, and peat. Astounding.",1
3,555,"\nThis 41 year old single cask was aged in a sherry butt and interacted magnificently with that wood. The nose offers peeled red apples, sultanas, honey, fudge, milk chocolate, and American cream soda. Balanced and harmonious throughout, the palate yields remarkably fresh orchard fruits, pipe tobacco, black pepper, and ginger. Long in the finish, with cinnamon drying to dark chocolate, and slightly smoky, unobtrusively tannic oak. A 1970s classic!",1
4,1965,"\nQuite herbal on the nose, with aromas of dried tarragon, parsley, and dill, along with cherry, milk chocolate, orange blossom, vanilla bean, and sweet tea. The palate is slightly grain-forward, but is full and well balanced, with orange, dried flowers, chamomile tea, lemon rind, and ginger. The finish offers cinnamon and milk chocolate against a note of candied lemon. Youthful and barrel-influenced, but admirably executed.",1


In [4]:
target = 'ratingCategory'
X = train.drop(columns=target)
y = train[target]

In [5]:
X

,id,description
0,1321,"\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133"
1,3861,"\nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)"
2,655,"\nThis release is a port version of Amrut’s Intermediate Sherry — a sort of port pipe sandwich. The spirit is matured in both unused casks and bourbon casks, then spends a few months in port pipes, and then returns to bourbon casks. The result is a Pink Floyd show of a whisky: vibrant, colorful, complex, and nearly too much. A blackcurrant and wispy, smoky nose gives way to an intense and bittersweet mix of chili, blackcurrant, oak, damson, dark chocolate, and peat. Astounding."
3,555,"\nThis 41 year old single cask was aged in a sherry butt and interacted magnificently with that wood. The nose offers peeled red apples, sultanas, honey, fudge, milk chocolate, and American cream soda. Balanced and harmonious throughout, the palate yields remarkably fresh orchard fruits, pipe tobacco, black pepper, and ginger. Long in the finish, with cinnamon drying to dark chocolate, and slightly smoky, unobtrusively tannic oak. A 1970s classic!"
4,1965,"\nQuite herbal on the nose, with aromas of dried tarragon, parsley, and dill, along with cherry, milk chocolate, orange blossom, vanilla bean, and sweet tea. The palate is slightly grain-forward, but is full and well balanced, with orange, dried flowers, chamomile tea, lemon rind, and ginger. The finish offers cinnamon and milk chocolate against a note of candied lemon. Youthful and barrel-influenced, but admirably executed."
...,...,...
4082,3342,"\nWhat lies beneath the surface of Dewar’s? Here, the blend is finished in virgin and first-fill bourbon casks especially charred, then scratched to enable deeper penetration. Thick wedge of vanilla and fudge with oaky accents, a dusty mouthfeel pulling in vanilla, light lemon, milk chocolate, and sweet toffee, ending on a cocoa powder note. Compared with regular White Label, this is less obviously sweet and creamy, adds complexity, and opens a bourbon connection. Certainly, this is up to scratch."
4083,3130,"\nAfter 6 to 7 years of maturation in bourbon casks, this spent several months in chardonnay white wine casks. The nose offers resinous wine notes, melon, grapefruit, and a hint of vanilla. Nutty on the palate, with orchard fruits, more vanilla, and milk chocolate. Peaches and a slightly oily note in the finish, with apples, nutmeg, and a sprinkling of black pepper."
4084,2811,"\nBright, delicate, and approachable. While not a showstopper, it delivers generous sweet fruit, almost candied red apple sweetness, along with floral and black cherry notes. On the palate, it shows modest structure, with notes of lemon and pistachio, along with gentle wisps of baking spice and sandalwood. A blend of two Kentucky straight rye whiskeys and two Kentucky straight bourbons."
4085,478,"\nI’m calling this the pitmaster’s dram: the nose has muscular peat smoke, prune juice, savory notes of barbecued brisket bark, venison charcuterie, and a hint of peppermint cream. The palate is a gluttonous feast of sweet dark fruits, milk chocolate, dried fig, plum, s

In [6]:
def clean_data(text):
    """
    Accepts a single text document and performs several regex substitutions in order to clean the document. 
    Parameters
    ----------
    text: string or object 
    Returns
    -------
    text: string or object
    """
    # order of operations - apply the expression from top to bottom
    email_regex = r"From: \S+@\S+"
    non_alpha = '[^a-zA-Z0-9]'
    multi_white_spaces = "[ ]{2,}"
    text = re.sub(email_regex, "", text)
    text = re.sub(non_alpha, ' ', text)
    text = re.sub(multi_white_spaces, " ", text)
    # apply case normalization 
    return text.lower().lstrip().rstrip()

In [7]:
X['description'] = X['description'].apply(clean_data)

In [8]:
X

,id,description
0,1321,sometimes when whisky is batched a few leftover barrels are returned to the warehouse canadian club recently pulled and vatted several of these from the 1970s acetone granny smith apples and fresh cut white cedar showcase this long age complex and spicy yet reserved this dram is ripe with strawberries canned pears cloves pepper and faint flowers then slightly pulling oak tannins distinct elegant and remarkably vibrant this ancient canadian club is anything but tired australia only a 133
1,3861,an uncommon exclusive bottling of a 6 year old cask strength malt light gold in color the nose is vegetal more peat bog than peat smoke with an undercurrent of pastry cream and rose it s an odd combination of aromas the entry is flavorful and inviting with smoked pineapple clove and rose peak smoke arrives in full force in the mid palate which drops the sweet and becomes spicy the finish is mostly smoke but with a pleasant minty coolness wyoming only
2,655,this release is a port version of amrut s intermediate sherry a sort of port pipe sandwich the spirit is matured in both unused casks and bourbon casks then spends a few months in port pipes and then returns to bourbon casks the result is a pink floyd show of a whisky vibrant colorful complex and nearly too much a blackcurrant and wispy smoky nose gives way to an intense and bittersweet mix of chili blackcurrant oak damson dark chocolate and peat astounding
3,555,this 41 year old single cask was aged in a sherry butt and interacted magnificently with that wood the nose offers peeled red apples sultanas honey fudge milk chocolate and american cream soda balanced and harmonious throughout the palate yields remarkably fresh orchard fruits pipe tobacco black pepper and ginger long in the finish with cinnamon drying to dark chocolate and slightly smoky unobtrusively tannic oak a 1970s classic
4,1965,quite herbal on the nose with aromas of dried tarragon parsley and dill along with cherry milk chocolate orange blossom vanilla bean and sweet tea the palate is slightly grain forward but is full and well balanced with orange dried flowers chamomile tea lemon rind and ginger the finish offers cinnamon and milk chocolate against a note of candied lemon youthful and barrel influenced but admirably executed
...,...,...
4082,3342,what lies beneath the surface of dewar s here the blend is finished in virgin and first fill bourbon casks especially charred then scratched to enable deeper penetration thick wedge of vanilla and fudge with oaky accents a dusty mouthfeel pulling in vanilla light lemon milk chocolate and sweet toffee ending on a cocoa powder note compared with regular white label this is less obviously sweet and creamy adds complexity and opens a bourbon connection certainly this is up to scratch
4083,3130,after 6 to 7 years of maturation in bourbon casks this spent several months in chardonnay white wine casks the nose offers resinous wine notes melon grapefruit and a hint of vanilla nutty on the palate with orchard fruits more vanilla and milk chocolate peaches and a slightly oily note in the finish with apples nutmeg and a sprinkling of black pepper
4084,2811,bright delicate and approachable while not a showstopper it delivers generous sweet fruit almost candied red apple sweetness along with floral and black cherry notes on the palate it shows modest structure with notes of lemon and pistachio along with gentle wisps of baking spice and sandalwood a blend of two kentucky straight rye whiskeys and two kentucky straight bourbons
4085,478,i m calling this the pitmaster s dram the nose has muscular peat smoke prune juice savory notes of barbecued brisket bark venison charcuterie and a hint of peppermint cream the palate is a gluttonous feast of sweet dark fruits milk chocolate dried fig plum sultanas and ginger sliding into baked orange and spicy gingerbread and wallowing luxuriously in chocolate and rich fruit a welcome addition to eifel s already impressive range 1 

### Define Pipeline Components

In [9]:
vect = ...
clf = ...

pipe = Pipeline([('vect', vect), ('clf', clf)])

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'Ellipsis' (type <class 'ellipsis'>) doesn't

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
subNumber = 0

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
lsi = ...
vect = ...
clf = ...

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?